For a given antenna size, find the solutions with minimal ambiguity
the doppler bandwidth can be reduced

1. find the optimal line in the timing diagram plane (based on antenna width), i.e., elevation 3-db angle
2. find all valid swaths close to the optimal line in the timing diagram plane. don't care about the nadir return, this can be discarded in the final image anywais.
3. make the valid swaths selectable by an order *n* identifying how far the point is from the optimal.


plot timing diag
plot optimal prf vs looking angle
todo test timingdiagram plotter here

In [1]:
# From Michelangelo Villano's hands on lab
# To open separate plot-windows outside the browser uncomment one of the following two lines
%matplotlib qt
#get_ipython().run_line_magic('matplotlib','qt5')

# To open a Plot-window within notebook with zoom/edit control uncomment one of the following two lines
# %matplotlib notebook
# get_ipython().run_line_magic('matplotlib','notebook')

# options are 'osx', 'qt4', 'qt5', 'gtk3', 'wx', 'qt', 'gtk', 'tk' , 'notebook' , 'inline'

In [2]:
## Dependencies
from libs.timing_diagram import *
from libs.design_functions import *
import numpy as np
import matplotlib.pyplot as plt
from libs.spherical_earth_geometry_radar import *
from libs.radartools.farField import UniformAperture
from libs.ambiguity_functions import *

In [3]:
# constants
c = 299792458

# radar parameters
dutycycle = 0.25  # duty cycle
h = 500e3  # height
wavelength = c / 10e9  # wavelength

# satellite speed
vs = orbital_speed(h)

# antenna size
Wa = 0.3  # antenna width in meters
La = 2  # antenna length in meters

# nadir duration in fractions of PRI for visualization in timing diagram
# note, it makes sense to use this fractional quantity as the nadir duration, if unfocused (e.g. saturated receiver), will be proportional to the impulse on time.
nadir_duration = 2 * dutycycle


In [4]:
# PRF axis
PRI = 1 / 7050
prf = np.linspace(1 / PRI - 4000, 1 / PRI + 4000, 100)

# plot
fig, ax = plt.subplots(1)
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False)
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h)
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)

C:\Users\menca\PycharmProjects\systemtradeoffs\libs\design_functions.py:42: RuntimeWarning: invalid value encountered in arccos
  beta = arccos(((re + h) ** 2 - re ** 2 + rs ** 2) / (2 * (re + h) * rs))
C:\Users\menca\PycharmProjects\systemtradeoffs\libs\design_functions.py:43: RuntimeWarning: invalid value encountered in arccos
  alpha = arccos(((re + h) ** 2 + re ** 2 - rs ** 2) / (2 * (re + h) * re))


(100.0, 300.0)

Finding the "optimal" PRF for each looking angle at each PRF -- Ground range broadside point
1. find the incidence angle from ground range
2. find the min max incidence angles of the swath extremes $\eta_{1,2} = \eta_i \pm \dfrac{\lambda}{W_A}$
3. find the slant range delta from $\eta_{1,2}$
4. find PRF from delta range

In [5]:
# 1 ground range axis in m
ground_range = np.linspace(0, 2000, 500) * 1000
# 2 incidence angle
slant_range = range_ground_to_slant(ground_range, h)
# loopy but functional
ground_range, eta = range_slant_to_ground(slant_range, h)  # the incidence angle is the second parameter returned
# 3 incidence range
eta1 = eta - wavelength / (2 * Wa)
eta2 = eta + wavelength / (2 * Wa)
slant_range_1, ground_range_1 = range_from_theta(eta1 * 180 / np.pi, h)
slant_range_2, ground_range_2 = range_from_theta(eta2 * 180 / np.pi, h)
# 4 slant range delta
slant_delta_range = slant_range_2 - slant_range_1
# 5 prf from slant range
pri = 2 * slant_delta_range / c
prf_opt = 1 / pri

C:\Users\menca\AppData\Local\Temp\ipykernel_21164\1203726107.py:16: RuntimeWarning: divide by zero encountered in divide
  prf_opt = 1 / pri


In [6]:
# Add to the plot
ax.plot(prf_opt, ground_range / 1000, 'red')

plt.show()

In [7]:
sr = range_ground_to_slant(200e3, h)
# loopy but functional
gr, eta = range_slant_to_ground(sr, h)  # the incidence angle is the second parameter returned
print(eta * 180 / np.pi, gr / 1000)

23.4734561406255 199.99999999999412


Find the minimum PRF line, i.e. the Doppler bandwidth approximated as
\begin{equation}
    B_D = \dfrac{2}{La} v_g
\end{equation}
with the ground velocity
\begin{equation}
 v_g = \dfrac{R_E \cos\Theta_E}{ R_E + h} v_s
\end{equation}
where
\begin{equation}
    \Theta_E = \dfrac{r_g}{R_E}
\end{equation}
$R_E$ is the Earth radius, $r_g$ the ground range, $v_s$ the satellite orbital speed


In [8]:
prf_min = 2 * ground_speed(ground_range, vs, h) / La

In [9]:
# Add to the plot
ax.plot(prf_min, ground_range / 1000, 'green')

plt.show()

In [10]:
range_from_theta(26.8)

(555062.2974338281, 232104.76792566918)

Find the swath given a point in the time diagram and plot it as a vetical line
i.e.
find closest end of transmission
fing closest start of transmission
plot a line (constant prf)

In [11]:
## input
rg_coordinate = 210e3  # m
prf_coordinate = 6800  # Hz

# maximum 'acceptable' ambiguity levels
AASR_max = - 20  # dB
RASR_max = -20  # dB

In [12]:
swath_prf = np.zeros(2)
swath_rg = np.zeros_like(swath_prf)
swath_rg[0], swath_prf[0] = last_end_of_transmission(rg_coordinate, prf_coordinate, dutycycle, h)
swath_rg[1], swath_prf[1] = next_start_of_transmission(rg_coordinate, prf_coordinate, dutycycle, h)

In [13]:
# total swath considering compression time
# 1 swath from pulse to pulse
swath_rs = range_ground_to_slant(swath_rg, h)
# 2 swath without 1 pulse period (half on each side)
swath_rs_compressed = swath_rs + np.array(
    [(1 / swath_prf[0] * dutycycle * c / 4), - 1 / swath_prf[1] * dutycycle * c / 4])
# 3 convert back to ground range
swath_rg_compressed, swath_eta_compressed = range_slant_to_ground(swath_rs_compressed, h)
# swath width in km
print('swath width on ground =', (swath_rg_compressed[-1] - swath_rg_compressed[0]) / 1000, 'km')

swath width on ground = 28.143342631575738 km


In [14]:
# add to plot
ax.plot(swath_prf, swath_rg / 1000, 'k')
ax.plot(swath_prf, swath_rg_compressed / 1000, 'P', color='k')
plt.show()

Compute the RASR and the AASR in the defined swath
1. Define a RadarGeometry centered at the center of the swath
2. adjust the looking angle to maximize NESZ
3. compute RASR and AASR
4. Plot RASR and  AASR over doppler bandwidth / processed bandwidth
5. Print NESZ at the swath center


In [15]:
# 1
# radar geometry class initialization
radar_geo = RadarGeometry()
radar_geo.set_rotation(30 * np.pi / 180, 0, 0)
radar_geo.set_initial_position(0, 0, h)
radar_geo.set_speed(vs)

# swath center incidence angle
rg_center, eta_center = range_slant_to_ground(np.average(swath_rs_compressed), h)
looking_angle = incidence_angle_to_looking_angle(eta_center, h)
radar_geo.set_rotation(looking_angle, 0, 0)
print('initial looking angle:', looking_angle * 180 / np.pi)

initial looking angle: 21.34185831339665


In [16]:
# 2
# Uniform aperture antenna initialization
uniap = UniformAperture(La, Wa, c / wavelength)
# Optimization error function (core SNR in spherical earth at the edges of the swath)
error_func = lambda b: snr_error(b, swath_eta_compressed[0], swath_eta_compressed[1], radar_geo, uniap)
# b is the looking angle of the radar
# Optimization using python native optimization methods
looking_angle_opt = fsolve(error_func, looking_angle, maxfev=100)
print('optimized looking angle:', looking_angle_opt[0] * 180 / np.pi)
# set the new looking angle
radar_geo.set_rotation(looking_angle_opt, 0, 0)

optimized looking angle: 21.4455424832387


(array([[ 1.],
        [ 0.],
        [-0.]]),
 array([[ 0.        ],
        [-0.93076549],
        [ 0.36561673]]),
 array([[ 0.        ],
        [-0.36561673],
        [-0.93076549]]))

In [17]:
# 3
# find the RASR over the observable range
# support axis
incidence_axis = np.linspace(swath_eta_compressed[0], swath_eta_compressed[1], 30)
slant_range_axis, ground_range_axis = range_from_theta(incidence_axis * 180 / np.pi, h)
# nominal Doppler bandwidth
Bd = 2 * ground_speed(rg_center, vs, h) / La * 0.5
rasr = RASR(radar_geo, uniap, incidence_axis, 1 / prf_coordinate, Bd, wavelength, vs, h, pbaroff=False)

  0%|          | 0/94 [00:00<?, ?it/s]C:\Users\menca\PycharmProjects\systemtradeoffs\libs\spherical_earth_geometry_radar.py:353: RuntimeWarning: invalid value encountered in arccos
  theta_e = np.arccos(cos_theta_e) * np.sign(incidence_mesh)  # to consider also incidence angles behind nadir
C:\Users\menca\PycharmProjects\systemtradeoffs\libs\ambiguity_functions.py:174: RuntimeWarning: invalid value encountered in divide
  Numer += np.where(sin(thetaj) != 0, Gint / (raxj ** 3 * sin(thetaj)), 0)
C:\Users\menca\PycharmProjects\systemtradeoffs\libs\ambiguity_functions.py:180: RuntimeWarning: invalid value encountered in divide
  Numer += np.where(sin(thetaj) != 0, Gint / (raxj ** 3 * sin(thetaj)), 0)
100%|██████████| 94/94 [00:03<00:00, 28.67it/s]


In [18]:
fig1, ax1 = plt.subplots(1)
ax1.plot(ground_range_axis / 1000, 10 * np.log10(rasr))
ax1.set_xlabel('ground range [km]')
ax1.set_ylabel('RASR [dB]')

Text(0, 0.5, 'RASR [dB]')

In [19]:
# AASR
doppler_undersampling_ratio = np.linspace(.01, 1, 15)
aasr = np.zeros_like(doppler_undersampling_ratio)
for ii in tqdm(range(len(aasr))):
    aasr[ii] = AASR(radar_geo, uniap, eta_center * 180 / np.pi, prf_coordinate, Bd * doppler_undersampling_ratio[ii],
                    wavelength, pbaroff=True)


 27%|██▋       | 4/15 [00:07<00:19,  1.81s/it]C:\Users\menca\PycharmProjects\systemtradeoffs\libs\spherical_earth_geometry_radar.py:290: RuntimeWarning: invalid value encountered in sqrt
  arg = ((lambda_c ** 2 * doppler_mesh ** 2 + np.sqrt(
100%|██████████| 15/15 [00:27<00:00,  1.87s/it]


In [20]:
fig2, ax2 = plt.subplots(1)
ax2.plot(doppler_undersampling_ratio, 10 * np.log10(aasr))
ax2.set_xlabel('Doppler undersampling ratio')
ax2.set_ylabel('AASR [dB]')

Text(0, 0.5, 'AASR [dB]')

In [21]:
# Core SNR
core_snr, azres = core_snr_spherical(radar_geo, uniap, eta_center, wavelength, vs, h, 1)
print(10 * np.log10(core_snr[0]), 'dB')

97.22490251632344 dB


Add a thresholding on RASR and AASR

In [22]:
# RASR threshold, numerical on the computed vector, no recalculation.
# find the swath with rasr smaller than RASR min
indexes = np.argwhere(rasr < 10 ** (RASR_max / 10))
ranges = ground_range_axis[indexes]
# differential integration
swath_rasr = 0
for ii in range(len(ranges) - 1):
    swath_rasr += ranges[-ii - 1] - ranges[-ii - 2]
if swath_rasr != 0:
    print("swath with acceptable RASR:", swath_rasr[0])
else:
    print("RASR exceeding max over all the swath")

swath with acceptable RASR: 17596.87320741039


In [23]:
# add to plot the RASR threshold
if swath_rasr != 0:
    ax1.scatter(ranges / 1000, np.ones_like(ranges) * RASR_max, marker='.', color='r',
                label=str('rg=' + str(np.round(swath_rasr[0] / 1000, 2)) + '[km]'))
    ax1.legend()

In [24]:
# AASR threshold linear interpolation between computed samples

indexes = np.argwhere(aasr < 10 ** (AASR_max / 10))
if len(indexes) != 0:
    undersampling = doppler_undersampling_ratio[max(indexes)]
    print(undersampling)
    if max(indexes) < len(aasr):
        # linear interpolation
        x = (AASR_max)
        x_0 = 10 * np.log10(aasr[max(indexes)])
        x_1 = 10 * np.log10(aasr[max(indexes) + 1])
        y_0 = doppler_undersampling_ratio[max(indexes)]
        y_1 = doppler_undersampling_ratio[max(indexes) + 1]
        undersampling = (y_0 * (x_1 - x) + y_1 * (x - x_0)) / (x_1 - x_0)
    print(undersampling)
else:
    undersampling = 0

[0.22214286]
[0.25009771]


In [25]:
ax2.scatter(undersampling, AASR_max, marker='o', color='r',
            label='undersampling ratio =' + str(np.round(undersampling[0], 2)))
ax2.legend()

In [26]:
1/0.25

4.0

In [31]:
# optimize processed doppler bandwidth for aasr
fractional = fsolve(
    lambda p: 10 * np.log10(AASR(radar_geo, uniap, eta_center * 180 / np.pi, prf_coordinate, Bd * p, wavelength, pbaroff=True)) - AASR_max, 1, maxfev=26)

C:\Users\menca\PycharmProjects\systemtradeoffs\venv\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The number of calls to function has reached maxfev = 26.
  warnings.warn(msg, RuntimeWarning)


In [30]:
print(fractional)

[0.23363243]


In [32]:
print(fractional)

[0.23477133]


In [ ]:
# the optimization method is slower and doesn't improve much the resolution